# RNN으로만든모델_테스트데이터_분석

In [23]:
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

In [4]:
test_data = pd.read_csv("./data/bank_app_reviews_test.csv")
test_data.head()

,리뷰일,평점,사용자리뷰,업체답변,은행명
0,2024-02-08,5,고경민계장님감사해요,"안녕하세요 최순녀 고객님. 칭찬 진심으로 감사드리며, 더욱 편리하고 안정적인 서비스...",우리
1,2023-07-24,5,저축목표피드 새로 생긴거 너무좋은데 분명 카테고리를 저축으로 했는데 왜 인식이 안되...,"신아​ 님, 안녕하세요? 뱅크샐러드 고객감동팀​입니다. 소중한 시간내어 고객센터에 ...",뱅크샐러드
2,2023-09-25,1,아니 이딴걸 편리하게 사용하는앱이라고 쳐만들엇나 이렇게 불편하게만든건 일부러그런거에...,안녕하세요. 우리은행입니다. 먼저 우리WON뱅킹 이용에 불편을 드려 죄송합니다. 보...,우리
3,2024-02-15,3,몇 년째 만족하며 사용중이라 조금식 개선되어거는 모습에 만족하며 사용중입니다. 하지...,안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드에 KB pay를 연결해 모든 자...,뱅크샐러드
4,2023-06-19,5,스타뱅킹을 사용 하고나서부터 편안해서 좋아요,"한송림 고객님, 안녕하세요! KB스타뱅킹을 이용해 주셔서 진심으로 감사드립니다. 앞...",국민


In [5]:
import re

def clean_text(text):
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s]','', text) #한글, 영문, 숫자
    cleaned = re.sub(r'\s+', ' ', cleaned) # 연속된 공백을 하나의 공백
    return cleaned.strip()

In [6]:
test_data['사용자리뷰'] = test_data['사용자리뷰'].apply(clean_text)
test_data['사용자리뷰']

0                                              고경민계장님감사해요
1       저축목표피드 새로 생긴거 너무좋은데 분명 카테고리를 저축으로 했는데 왜 인식이 안되...
2         아니 이딴걸 편리하게 사용하는앱이라고 쳐만들엇나 이렇게 불편하게만든건 일부러그런거에요
3       몇 년째 만족하며 사용중이라 조금식 개선되어거는 모습에 만족하며 사용중입니다 하지만...
4                                스타뱅킹을 사용 하고나서부터 편안해서 좋아요
                              ...                        
9529    만보기 이벤트는 실망스러워요 후기 말투 다 똑같고 사기 맞죠 양심이 참 정직하게 확...
9530                   기능이 많아 다 사용해보진 못 했지만 대체적으로 편한거 같아요
9531                                                편리하네요
9532                                            사용하기 편리해요
9533    너무 후져서 오랜만에 어플이용하다가 욕했답니다 인증방식이 2010년대에 머물러계심 ...
Name: 사용자리뷰, Length: 9534, dtype: object

In [7]:
test_data['is_good'] = test_data['평점'].apply(lambda x: 1 if x >=4 else 0)
test_data['is_good']

0       1
1       1
2       0
3       0
4       1
       ..
9529    0
9530    1
9531    1
9532    1
9533    0
Name: is_good, Length: 9534, dtype: int64

## train에서 사용했던 tokenizer 불러와서 원핫인코딩 수행

In [8]:
mecab = Mecab()

In [10]:
tokenized_docs = test_data['사용자리뷰'].apply(mecab.morphs)
tokenized_docs[0]

['고경민', '계장', '님', '감사', '해요']

In [11]:
import joblib

In [12]:
token = joblib.load("./model/bank_app_tokenizer.joblib")

In [14]:
x = token.texts_to_sequences(tokenized_docs)
print(x[0])

[6248, 327, 111, 71]


## train에서 사용했던 패딩 길이(모델에 넣을 컬럼의 수)

In [17]:
max_length = joblib.load("./model/bank_app_max_length.joblib")
print(max_length)

302


In [18]:
X_padded = pad_sequences(x, maxlen=max_length, padding='post')
print(X_padded[1])

[ 216  717  370 1524   39   44    8  148 1025  724   27 1033   31   43
   14   51  117    3    6    9    4  861  117    9  414  224  112    9
  164  409 2261 1336   20    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [21]:
y = test_data['is_good']
y

0       1
1       1
2       0
3       0
4       1
       ..
9529    0
9530    1
9531    1
9532    1
9533    0
Name: is_good, Length: 9534, dtype: int64

# 모델 불러와서 예측하고 결과 비교하기

In [24]:
birnn_bast = load_model("./model/bank_app_review_birnn.keras")
cnn_lstm_best = load_model("./model/bank_app_review_lstm_cnn.keras")
attn_best = load_model("./model/bank_app_review_attn_model.keras")


I0000 00:00:1747887871.407992    2944 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1347 MB memory:  -> device: 0, name: NVIDIA GeForce MX450, pci bus id: 0000:01:00.0, compute capability: 7.5


In [25]:
birnn_pred = birnn_bast.predict(X_padded)
cnn_lstm_pred = cnn_lstm_best.predict(X_padded)
attn_pred = attn_best.predict(X_padded)

I0000 00:00:1747887942.670112    5796 service.cc:152] XLA service 0x55bc2ba942b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747887942.670429    5796 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce MX450, Compute Capability 7.5
2025-05-22 13:25:42.818737: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747887943.096111    5796 cuda_dnn.cc:529] Loaded cuDNN version 90300


  5/298 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step

I0000 00:00:1747887945.241081    5796 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


298/298 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step
298/298 ━━━━━━━━━━━━━━━━━━━━ 20s 46ms/step
298/298 ━━━━━━━━━━━━━━━━━━━━ 39s 120ms/step


In [26]:
birnn_pred = pd.DataFrame(birnn_pred)
cnn_lstm_pred = pd.DataFrame(cnn_lstm_pred)
attn_pred = pd.DataFrame(attn_pred)

In [32]:
y = pd.DataFrame(y)

In [34]:
birnn_result = y.join(birnn_pred)
cnn_lstm_result = y.join(cnn_lstm_pred)
attn_result = y.join(attn_pred)

In [37]:
birnn_result.loc[:, 0] = birnn_result.loc[:, 0].apply(lambda x: 1 if x>0.5 else 0)
cnn_lstm_result.loc[:, 0] = cnn_lstm_result.loc[:, 0].apply(lambda x: 1 if x>0.5 else 0)
attn_result.loc[:, 0] = attn_result.loc[:, 0].apply(lambda x: 1 if x>0.5 else 0)

In [49]:
birnn_result

,is_good,0
0,1,1.0
1,1,1.0
2,0,0.0
3,0,1.0
4,1,1.0
...,...,...
9529,0,0.0
9530,1,1.0
9531,1,1.0
9532,1,1.0


In [50]:
cnn_lstm_result

,is_good,0
0,1,1.0
1,1,0.0
2,0,0.0
3,0,1.0
4,1,1.0
...,...,...
9529,0,0.0
9530,1,1.0
9531,1,1.0
9532,1,1.0


In [51]:
attn_result

,is_good,0
0,1,1.0
1,1,1.0
2,0,0.0
3,0,1.0
4,1,1.0
...,...,...
9529,0,0.0
9530,1,1.0
9531,1,1.0
9532,1,1.0


In [44]:
from sklearn.metrics import classification_report

In [45]:
print(classification_report(birnn_result['is_good'], birnn_result[0]))

              precision    recall  f1-score   support

           0       0.89      0.85      0.87      3862
           1       0.90      0.92      0.91      5672

    accuracy                           0.90      9534
   macro avg       0.89      0.89      0.89      9534
weighted avg       0.90      0.90      0.90      9534



In [52]:
# 훈련시간과 예측시간 모두 빨랐음 -> 정확도가 비슷하다고 가정하는 경우 빠르게 처리 가능한 모델 사용하는 것이 맞다
print(classification_report(cnn_lstm_result['is_good'], cnn_lstm_result[0]))

              precision    recall  f1-score   support

           0       0.86      0.89      0.87      3862
           1       0.93      0.90      0.91      5672

    accuracy                           0.90      9534
   macro avg       0.89      0.90      0.89      9534
weighted avg       0.90      0.90      0.90      9534



In [47]:
print(classification_report(attn_result['is_good'], attn_result[0]))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88      3862
           1       0.92      0.91      0.91      5672

    accuracy                           0.90      9534
   macro avg       0.89      0.90      0.90      9534
weighted avg       0.90      0.90      0.90      9534



## evaluate

In [55]:
%%time
birnn_bast.evaluate(X_padded, test_data['is_good'])

298/298 ━━━━━━━━━━━━━━━━━━━━ 43s 103ms/step - accuracy: 0.9002 - auc: 0.9509 - loss: 0.2743
CPU times: user 38.2 s, sys: 5.12 s, total: 43.3 s
Wall time: 43.9 s


[0.2772158086299896, 0.896265983581543, 0.9501931071281433]

In [56]:
%%time
cnn_lstm_best.evaluate(X_padded, test_data['is_good'])

298/298 ━━━━━━━━━━━━━━━━━━━━ 41s 106ms/step - accuracy: 0.8996 - auc: 0.9521 - loss: 0.2677
CPU times: user 37.8 s, sys: 9.17 s, total: 47 s
Wall time: 41.6 s


[0.26952075958251953, 0.8961610794067383, 0.9519997835159302]

In [57]:
%%time
attn_best.evaluate(X_padded, test_data['is_good'])

298/298 ━━━━━━━━━━━━━━━━━━━━ 46s 135ms/step - accuracy: 0.9022 - auc: 0.9586 - loss: 0.2488
CPU times: user 37.2 s, sys: 7.64 s, total: 44.8 s
Wall time: 46.4 s


[0.25195175409317017, 0.8985735177993774, 0.9580655097961426]